In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\python3\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\re19017\appdata\local\temp\pip-req-build-iuo2sz6x
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369272 sha256=b20432469e7d744f9f195e3b0fc8e4d9580ca12d404365320c9c3b191cea2be4
  Stored in directory: C:\Users\re19017\AppData\Local\Temp\pip-ephem-wheel-cache-k4vv3yen\wheels\c8\e4\e1\11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip


  Running command git clone -q https://github.com/openai/CLIP.git 'C:\Users\re19017\AppData\Local\Temp\pip-req-build-iuo2sz6x'
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\python3\python.exe -m pip install --upgrade pip' command.


In [1]:
import os, os.path
import numpy as np

In [2]:
import torch
import clip
from PIL import Image

In [4]:
import pandas as pd
dataset = pd.read_csv('week5.csv')
dataset.head()

,Image_Description,Link,Article_Title,Article_URL,Article_Text,Keywords
0,Rapid delivery's rapid rise has led to tension...,https://static.euronews.com/articles/stories/0...,Dutch cities temporarily banned 10-minute deli...,https://www.euronews.com/next/2022/02/24/dutch...,"['Noise, reckless cycling and blacked-out wind...","['company', 'shop', 'freeze', 'rotterdam', 'da..."
1,Start-up conference Step in Dubai Internet City.,https://static.euronews.com/articles/stories/0...,How Dubai Internet City is becoming a hub for ...,https://www.euronews.com/next/2022/02/23/how-d...,"['Khazna, one of the largest data centre infra...","['khazna', 'centre', 'internet', 'data', 'incr..."
2,Meta CEO Mark Zuckerberg said the company's co...,https://static.euronews.com/articles/stories/0...,"Ahoy, Metamates! Meta continues rebranding eff...",https://www.euronews.com/next/2022/02/16/ahoy-...,"[""Employees at Facebook's parent company Meta ...","['employee', 'company', 'privacy', 'use', 'met..."
3,Carolyn Kaster/AP Photo,https://static.euronews.com/articles/stories/0...,"Ahoy, Metamates! Meta continues rebranding eff...",https://www.euronews.com/next/2022/02/16/ahoy-...,"[""Employees at Facebook's parent company Meta ...","['employee', 'company', 'privacy', 'use', 'met..."
4,Choosing a green pension could be better for t...,https://static.euronews.com/articles/stories/0...,Green pensions could be the ‘most powerful wea...,https://www.euronews.com/green/2022/02/15/gree...,['Switching to a sustainable pension could be ...,"['company', 'invest', 'use', 'matter', 'pensio..."


In [5]:
import urllib
import numpy as np
import cv2
import time

theImages = []
theKeywords = [] 
sample = dataset.sample(20)
for imagepath, key in zip( sample["Link"], sample['Keywords'] ) :
    #print(\"Image: \", imagepath)
    req = urllib.request.urlopen(imagepath)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    im = cv2.imdecode(arr, -1)
    theImages.append(im)
    theKeywords.append(key)
    time.sleep(3)

filePaths = []
for i in range(len(theImages)):
    if theImages[i] is not None:
        fn = 'images/' + str(i) + '.png'
        filePaths.append(fn)
        #print(fn, theImages[i])
        cv2.imwrite(fn, theImages[i])
        

In [16]:
import re
re.findall("\w+", theKeywords[0]) 

['child',
 'furniture',
 'company',
 'recycle',
 'vanbriel',
 'product',
 'planet',
 'business',
 'joris',
 'toy']

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

keys = re.findall("\w+", theKeywords[0]) 
keys.append("person")
image = preprocess(Image.open(filePaths[0])).unsqueeze(0).to(device)
text = clip.tokenize(keys).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.01665869 0.19420391 0.02826662 0.00902828 0.28671438 0.08108789
  0.03962763 0.02636237 0.2253054  0.0331996  0.05954532]]


In [21]:
np.max(probs)
np.where(probs[0] == probs[0].max())[0][0]
theText = ["a person", "a dog", "a cat"]
theText[np.where(probs[0] == probs[0].max())[0][0]]

'a person'

In [23]:
imageFilenames = []
for f in os.listdir('images'):
    imageFilenames.append('images/'+f)
    
imageFilenames.sort()
imageFilenames

['images/0.png',
 'images/1.png',
 'images/10.png',
 'images/100.png',
 'images/101.png',
 'images/102.png',
 'images/103.png',
 'images/104.png',
 'images/105.png',
 'images/106.png',
 'images/107.png',
 'images/108.png',
 'images/109.png',
 'images/11.png',
 'images/110.png',
 'images/111.png',
 'images/112.png',
 'images/113.png',
 'images/114.png',
 'images/115.png',
 'images/116.png',
 'images/12.png',
 'images/13.png',
 'images/14.png',
 'images/15.png',
 'images/16.png',
 'images/17.png',
 'images/18.png',
 'images/19.png',
 'images/2.png',
 'images/21.png',
 'images/22.png',
 'images/23.png',
 'images/24.png',
 'images/25.png',
 'images/26.png',
 'images/27.png',
 'images/28.png',
 'images/29.png',
 'images/30.png',
 'images/31.png',
 'images/32.png',
 'images/33.png',
 'images/34.png',
 'images/35.png',
 'images/36.png',
 'images/37.png',
 'images/38.png',
 'images/39.png',
 'images/4.png',
 'images/40.png',
 'images/41.png',
 'images/42.png',
 'images/43.png',
 'images/44.png

In [19]:
#theText = ["a person", "a building", "animated", "a vehicle", "a landscape"]
#text = clip.tokenize(theText).to(device)


for f, k in zip( filePaths, theKeywords):
    keys = re.findall("\w+", k) 
    keys.append("person")
    image = preprocess(Image.open(f)).unsqueeze(0).to(device)
    text = clip.tokenize(keys).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("Filename:", f, "    Label probs:", probs)
    print("Label: ", keys[np.where(probs[0] == probs[0].max())[0][0]]) 
    print()

Filename: images/0.png     Label probs: [[0.01665869 0.19420391 0.02826662 0.00902828 0.28671438 0.08108789
  0.03962763 0.02636237 0.2253054  0.0331996  0.05954532]]
Label:  vanbriel

Filename: images/1.png     Label probs: [[0.17202117 0.19067247 0.09597183 0.17853592 0.05237311 0.17456755
  0.09446716 0.04139081]]
Label:  entrepreneur

Filename: images/2.png     Label probs: [[0.33175248 0.03803428 0.00589781 0.008564   0.5963856  0.00891118
  0.01045471]]
Label:  shopper

Filename: images/3.png     Label probs: [[9.72401910e-03 4.86902078e-04 9.54297304e-01 5.21288021e-03
  9.79406293e-03 1.45723615e-02 5.91252465e-03]]
Label:  ireland

Filename: images/4.png     Label probs: [[0.05762423 0.0862273  0.05676711 0.0344808  0.07155424 0.13480136
  0.03246328 0.12635887 0.07164997 0.2272038  0.10086897]]
Label:  mol

Filename: images/5.png     Label probs: [[0.01226546 0.30095688 0.04362405 0.07602143 0.08810082 0.28214264
  0.07414858 0.01326635 0.00307582 0.10639806]]
Label:  company